### 部署相关库的安装
    如果想要大规模抓取数据，那么一定会用到分布式爬虫。对于分布式爬虫来说，我们需要多台主机，每台主机有多个爬虫任务，但是源代码其实只有一份。此时我们需要做的就是将一份代码同时部署到多台主机上来协同运行，那么怎么去部署就是另一个值得思考的问题。对于 Scrapy 来说，它有一个扩展组件，叫作 Scrapyd，我们只需要安装该扩展组件，即可远程管理 Scrapy 任务，包括部署源码、启动任务、监听任务等。另外，还有 Scrapyd-Client 和 Scrapyd API 来帮助我们更方便地完成部署和监听操作。
    另外，还有一种部署方式，那就是 Docker 集群部署。我们只需要将爬虫制作为 Docker 镜像，只要主机安装了 Docker，就可以直接运行爬虫，而无需再去担心环境配置、版本问题。

#### 1.Docker 的安装
    Docker 是一种容器技术，可以将应用和环境等进行打包，形成一个独立的、类似于 iOS 的 App 形式的 “应用”。这个应用可以直接被分发到任意一个支持 Docker 的环境中，通过简单的命令即可启动运行。Docker 是一种最流行的容器化实现方案，和虚拟化技术类似，它极大地方便了应用服务的部署；又与虚拟化技术不同，它以一种更轻量的方式实现了应用服务的打包。使用 Docker，可以让每个应用彼此相互隔离，在同一台机器上同时运行多个应用，不过它们彼此之间共享同一个操作系统。Docker 的优势在于，它可以在更细的粒度上进行资源管理，也比虚拟化技术更加节约资源。对于爬虫来说，如果我们需要大规模部署爬虫系统的话，用 Docker 会大大提高效率。工欲善其事，必先利其器。

    官方网站：https://www.docker.com
    GitHub：https://github.com/docker
    Docker Hub：https://hub.docker.com
    官方文档：https://docs.docker.com
    DaoCloud：http://www.daocloud.io
    中文社区：http://www.docker.org.cn
    中文教程：http://www.runoob.com/docker/docker-tutorial.html
    推荐书籍：https://yeasy.gitbooks.io/docker_practice

#### 2.Scrapyd 的安装
    Scrapyd 是一个用于部署和运行 Scrapy 项目的工具，有了它，我们可以将写好的 Scrapy 项目上传到云主机并通过 API 来控制它的运行。既然是 Scrapy 项目部署，基本上都使用 Linux 主机，所以本节的安装是针对于 Linux 主机的。

    GitHub：https://github.com/scrapy/scrapyd
    PyPi：https://pypi.python.org/pypi/scrapyd
    官方文档：https://scrapyd.readthedocs.io
    
    安装：pip install scrapyd
    配置：安装完毕之后，需要新建一个配置文件 /etc/scrapyd/scrapyd.conf，Scrapyd 在运行的时候会读取此配置文件。在 Scrapyd 1.2 版本之后，不会自动创建该文件，需要我们自行添加。首先，执行如下命令新建文件：
    sudo mkdir /etc/scrapyd  
    sudo vi /etc/scrapyd/scrapyd.conf
    
    接着写入如下内容：
        [scrapyd]  
        eggs_dir    = eggs  
        logs_dir    = logs  
        items_dir   =  
        jobs_to_keep = 5  
        dbs_dir     = dbs  
        max_proc    = 0  
        max_proc_per_cpu = 10  
        finished_to_keep = 100  
        poll_interval = 5.0  
        bind_address = 0.0.0.0  
        http_port   = 6800  
        debug       = off  
        runner      = scrapyd.runner  
        application = scrapyd.app.application  
        launcher    = scrapyd.launcher.Launcher  
        webroot     = scrapyd.website.Root  

        [services]  
        schedule.json     = scrapyd.webservice.Schedule  
        cancel.json       = scrapyd.webservice.Cancel  
        addversion.json   = scrapyd.webservice.AddVersion  
        listprojects.json = scrapyd.webservice.ListProjects  
        listversions.json = scrapyd.webservice.ListVersions  
        listspiders.json  = scrapyd.webservice.ListSpiders  
        delproject.json   = scrapyd.webservice.DeleteProject  
        delversion.json   = scrapyd.webservice.DeleteVersion  
        listjobs.json     = scrapyd.webservice.ListJobs  
        daemonstatus.json = scrapyd.webservice.DaemonStatus
        
    配置文件的内容可以参见官方文档 https://scrapyd.readthedocs.io/en/stable/config.html#example-configuration-file。这里的配置文件有所修改，其中之一是 max_proc_per_cpu 官方默认为 4，即一台主机每个 CPU 最多运行 4 个 Scrapy 任务，在此提高为 10。另外一个是 bind_address，默认为本地 127.0.0.1，在此修改为 0.0.0.0，以使外网可以访问。
    
    后台运行：Scrapyd 是一个纯 Python 项目，这里可以直接调用它来运行。为了使程序一直在后台运行，Linux 和 Mac 可以使用如下命令：(scrapyd> /dev/null &)

    这样 Scrapyd 就会在后台持续运行了，控制台输出直接忽略。当然，如果想记录输出日志，可以修改输出目标，如：(scrapyd> ~/scrapyd.log &)
    此时会将 Scrapyd 的运行结果输出到～/scrapyd.log 文件中。当然也可以使用 screen、tmux、supervisor 等工具来实现进程守护。